Passage à une fonction


In [9]:
import pandas as pd
def transformation(numeroDossier, dossier):
    df= pd.read_csv(dossier, sep='\t')
    df.columns=['numéro','annotation','detail']
    df_ann=df['annotation'].convert_dtypes(convert_string=True)
    df['annotation']=df_ann.str.split(expand=True)[0]
    df.pop('numéro')
    df=df.pivot(columns='annotation')
    df_trait=df['detail'][['traitement']].dropna()
    df_path=df['detail'][['pathologie']].dropna()
    df_exam=df['detail'][['examen']].dropna()
    #df_trait['annotation'].replace(value=numeroDossier)
    print(df_trait)
    
    

In [10]:
transformation(numeroDossier=1,dossier='filepdf-5-cas.ann')

annotation            traitement
0           pénectomie partielle
61                    amputation


In [ ]:
import plotly as plt
import dash